In [18]:
import rasterio
from rasterio.transform import rowcol
from pyproj import Proj, Transformer

In [23]:
def get_raster_value_tif(lat, lon, raster_path):
    """
    Fetches the raster value at the given latitude and longitude.

    Parameters:
    lat (float): Latitude coordinate
    lon (float): Longitude coordinate
    raster_path (str): File path to the raster (.tif) file

    Returns:
    float: Raster value at the given coordinate
    """
    with rasterio.open(raster_path) as dataset:
        # Get raster coordinate reference system (CRS)
        raster_crs = dataset.crs
    

        transformer = Transformer.from_crs("EPSG:4326", raster_crs, always_xy=True)
        x, y = transformer.transform(lon, lat)

        row, col = rowcol(dataset.transform, x, y)

        value = dataset.read(1)[row, col]
    
    return value



In [24]:
raster_value = get_raster_value_tif(28.6139, 77.2090, "/Users/preitish/agrihackathon/FilteredNDVI.tif")
print(raster_value)

76


In [27]:
import rasterio
from rasterio.transform import rowcol
from pyproj import Transformer, CRS
import os

def get_raster_values(lat, lon, raster_paths, raster_crs="EPSG:32643"):
    """
    Fetches raster values at the given latitude and longitude for multiple raster files.

    Parameters:
    lat (float): Latitude coordinate
    lon (float): Longitude coordinate
    raster_paths (list): List of file paths to raster (.tif or .asc) files
    raster_crs (str): CRS of the raster files (default: "EPSG:32643")

    Returns:
    dict: Dictionary with filenames as keys and raster values as values
    """
    results = {}

    for raster_path in raster_paths:
        try:
            with rasterio.open(raster_path) as dataset:
                # Use dataset's CRS if available; otherwise, use the given CRS
                dataset_crs = dataset.crs if dataset.crs else CRS.from_string(raster_crs)

                # Convert latitude and longitude to raster coordinates
                transformer = Transformer.from_crs("EPSG:4326", dataset_crs, always_xy=True)
                x, y = transformer.transform(lon, lat)

                # Get row/col indices
                row, col = rowcol(dataset.transform, x, y)

                # Read raster value at the specified location
                value = dataset.read(1)[row, col]

                # Extract filename without extension and store the value
                file_name = os.path.splitext(os.path.basename(raster_path))[0]
                results[file_name] = value

        except Exception as e:
            results[os.path.basename(raster_path)] = f"Error: {str(e)}"

    return results

# Example usage with multiple files
raster_files = [
    "/Users/preitish/agrihackathon/GlobalNDVI.tif",
    "/Users/preitish/agrihackathon/FilteredNDVI.tif",
    "/Users/preitish/agrihackathon/LocalNDVI.tif",
    "/Users/preitish/agrihackathon/surfacewaterlevelproducts.tif",
    "/Users/preitish/agrihackathon/vegetationFraction.tif"
]

raster_values = get_raster_values(28.6139, 77.2090, raster_files, "EPSG:32643")

# Print the results
for key, value in raster_values.items():
    print(f"{key}: {value}")

GlobalNDVI: 123
FilteredNDVI: 76
LocalNDVI: 250
surfacewaterlevelproducts: 20
vegetationFraction: 250


In [32]:
import rasterio
from rasterio.transform import rowcol
from pyproj import Transformer, CRS
import os

def get_raster_values(lat, lon, raster_paths, raster_crs="EPSG:32643"):
    """
    Fetches raster values at the given latitude and longitude for multiple .asc raster files.

    Parameters:
    lat (float): Latitude coordinate
    lon (float): Longitude coordinate
    raster_paths (list): List of file paths to .asc raster files
    raster_crs (str): CRS of the raster files (default: "EPSG:32643")

    Returns:
    dict: Dictionary with filenames as keys and raster values as values
    """
    results = {}

    for raster_path in raster_paths:
        try:
            with rasterio.open(raster_path) as dataset:
                # If the dataset has a CRS, use it; otherwise, use the provided raster_crs
                dataset_crs = dataset.crs if dataset.crs else CRS.from_string(raster_crs)

                # Convert latitude and longitude to raster coordinates
                transformer = Transformer.from_crs("EPSG:4326", dataset_crs, always_xy=True)
                x, y = transformer.transform(lon, lat)

                # Get row/col indices
                row, col = rowcol(dataset.transform, x, y)

                # Read raster value at the specified location
                value = dataset.read(1)[row, col]

                # Extract filename without extension and store the value
                file_name = os.path.splitext(os.path.basename(raster_path))[0]
                results[file_name] = value

        except Exception as e:
            results[os.path.basename(raster_path)] = f"Error: {str(e)}"

    return results

# Example usage with multiple .asc files
asc_files = [
    "/Users/preitish/agrihackathon/kharif.asc",
    "/Users/preitish/agrihackathon/rabi.asc"
]

raster_values = get_raster_values(14.523144351233627, 89.30036236779829, asc_files, "EPSG:32643")

# Print the results
for key, value in raster_values.items():
    print(f"{key}: {value}")

kharif: 6124
rabi: 130
